# Chapter 9-3: 미니프로젝트 - 일기장 프로그램

## 프로젝트 개요
- 파일 입출력을 활용한 개인 일기장 프로그램
- JSON 형식으로 데이터 저장
- 검색, 통계, 백업 기능 구현
- 사용자 친화적 메뉴 시스템

In [ ]:
import json
import os
from datetime import datetime, date
from pathlib import Path

class DiaryManager:
    """일기장 관리 클래스"""
    
    def __init__(self, data_file='diary_data.json'):
        self.data_file = data_file
        self.entries = self.load_data()
    
    def load_data(self):
        """데이터 파일 로드"""
        try:
            with open(self.data_file, 'r', encoding='utf-8') as f:
                return json.load(f)
        except FileNotFoundError:
            return {}
    
    def save_data(self):
        """데이터 파일 저장"""
        with open(self.data_file, 'w', encoding='utf-8') as f:
            json.dump(self.entries, f, ensure_ascii=False, indent=2)
    
    def add_entry(self, content, mood='보통'):
        """일기 추가"""
        today = date.today().isoformat()
        entry = {
            'content': content,
            'mood': mood,
            'created_at': datetime.now().isoformat(),
            'word_count': len(content.split())
        }
        self.entries[today] = entry
        self.save_data()
        return f"{today} 일기가 저장되었습니다."
    
    def get_entry(self, date_str):
        """특정 날짜 일기 조회"""
        return self.entries.get(date_str)
    
    def search_entries(self, keyword):
        """키워드로 일기 검색"""
        results = []
        for date_str, entry in self.entries.items():
            if keyword.lower() in entry['content'].lower():
                results.append((date_str, entry))
        return results
    
    def get_statistics(self):
        """일기 통계"""
        if not self.entries:
            return None
        
        total_entries = len(self.entries)
        total_words = sum(entry['word_count'] for entry in self.entries.values())
        avg_words = total_words / total_entries
        
        mood_count = {}
        for entry in self.entries.values():
            mood = entry.get('mood', '보통')
            mood_count[mood] = mood_count.get(mood, 0) + 1
        
        return {
            'total_entries': total_entries,
            'total_words': total_words,
            'avg_words': round(avg_words, 1),
            'mood_distribution': mood_count
        }
    
    def export_to_text(self, filename='diary_export.txt'):
        """텍스트 파일로 내보내기"""
        with open(filename, 'w', encoding='utf-8') as f:
            f.write("=== 내 일기장 ===\n\n")
            for date_str in sorted(self.entries.keys()):
                entry = self.entries[date_str]
                f.write(f"날짜: {date_str}\n")
                f.write(f"기분: {entry.get('mood', '보통')}\n")
                f.write(f"내용:\n{entry['content']}\n")
                f.write("-" * 50 + "\n\n")
        return f"일기가 {filename}으로 내보내졌습니다."

# 일기장 매니저 생성
diary = DiaryManager()

print("📔 일기장 프로그램이 시작되었습니다!")
print(f"현재 저장된 일기 수: {len(diary.entries)}개")

In [ ]:
# 일기 작성 데모
print("=== 일기 작성 데모 ===")

# 샘플 일기들 추가
sample_entries = [
    ("오늘은 파이썬 공부를 했다. 파일 입출력이 생각보다 재미있었다. 특히 JSON 처리가 유용할 것 같다.", "좋음"),
    ("친구들과 카페에서 프로젝트 회의를 했다. 아이디어가 많이 나와서 기분이 좋았다.", "매우 좋음"),
    ("비가 와서 집에만 있었다. 독서를 하며 조용한 시간을 보냈다.", "보통")
]

for i, (content, mood) in enumerate(sample_entries):
    # 가상의 날짜로 저장 (실제로는 오늘 날짜가 사용됨)
    import datetime
    test_date = (datetime.date.today() - datetime.timedelta(days=i)).isoformat()
    diary.entries[test_date] = {
        'content': content,
        'mood': mood,
        'created_at': datetime.datetime.now().isoformat(),
        'word_count': len(content.split())
    }

diary.save_data()
print(f"샘플 일기 {len(sample_entries)}개가 추가되었습니다.")

# 일기 목록 출력
print("\n=== 저장된 일기 목록 ===")
for date_str in sorted(diary.entries.keys(), reverse=True):
    entry = diary.entries[date_str]
    content_preview = entry['content'][:30] + "..." if len(entry['content']) > 30 else entry['content']
    print(f"📅 {date_str} ({entry['mood']}) - {content_preview}")

In [ ]:
# 검색 기능 데모
print("=== 일기 검색 기능 ===")

search_keywords = ['파이썬', '친구', '비']
for keyword in search_keywords:
    results = diary.search_entries(keyword)
    print(f"\n'{keyword}' 검색 결과: {len(results)}건")
    for date_str, entry in results:
        print(f"  📅 {date_str}: {entry['content'][:40]}...")

# 통계 조회
print("\n=== 일기 통계 ===")
stats = diary.get_statistics()
if stats:
    print(f"📊 총 일기 수: {stats['total_entries']}개")
    print(f"📝 총 단어 수: {stats['total_words']}개")
    print(f"📈 평균 단어 수: {stats['avg_words']}개")
    print("\n😊 기분별 분포:")
    for mood, count in stats['mood_distribution'].items():
        percentage = (count / stats['total_entries']) * 100
        print(f"  {mood}: {count}개 ({percentage:.1f}%)")

# 텍스트 파일로 내보내기
export_result = diary.export_to_text('my_diary_backup.txt')
print(f"\n💾 {export_result}")

In [ ]:
# 메인 프로그램 (인터랙티브 버전)
def run_diary_app():
    """일기장 앱 실행"""
    diary = DiaryManager()
    
    while True:
        print("\n" + "="*40)
        print("📔 나의 일기장")
        print("="*40)
        print("1. 오늘 일기 쓰기")
        print("2. 일기 읽기")
        print("3. 일기 검색")
        print("4. 통계 보기")
        print("5. 백업하기")
        print("0. 종료")
        print("="*40)
        
        choice = input("선택하세요: ").strip()
        
        if choice == '1':
            print("\n📝 오늘의 일기를 작성해주세요:")
            content = input("내용: ")
            if content:
                moods = ['매우 나쁨', '나쁨', '보통', '좋음', '매우 좋음']
                print(f"기분을 선택하세요: {', '.join(f'{i+1}.{m}' for i, m in enumerate(moods))}")
                try:
                    mood_idx = int(input("번호: ")) - 1
                    mood = moods[mood_idx] if 0 <= mood_idx < len(moods) else '보통'
                except:
                    mood = '보통'
                
                result = diary.add_entry(content, mood)
                print(f"✅ {result}")
        
        elif choice == '2':
            date_str = input("날짜를 입력하세요 (YYYY-MM-DD): ")
            entry = diary.get_entry(date_str)
            if entry:
                print(f"\n📅 {date_str}의 일기")
                print(f"기분: {entry.get('mood', '보통')}")
                print(f"내용: {entry['content']}")
                print(f"단어 수: {entry['word_count']}개")
            else:
                print("❌ 해당 날짜의 일기가 없습니다.")
        
        elif choice == '3':
            keyword = input("검색할 키워드: ")
            results = diary.search_entries(keyword)
            if results:
                print(f"\n🔍 '{keyword}' 검색 결과: {len(results)}건")
                for date_str, entry in results:
                    preview = entry['content'][:50] + "..." if len(entry['content']) > 50 else entry['content']
                    print(f"  📅 {date_str}: {preview}")
            else:
                print("❌ 검색 결과가 없습니다.")
        
        elif choice == '4':
            stats = diary.get_statistics()
            if stats:
                print(f"\n📊 일기장 통계")
                print(f"총 일기: {stats['total_entries']}개")
                print(f"총 단어: {stats['total_words']}개")
                print(f"평균 단어: {stats['avg_words']}개")
                print("기분 분포:")
                for mood, count in stats['mood_distribution'].items():
                    print(f"  {mood}: {count}개")
            else:
                print("❌ 통계를 생성할 일기가 없습니다.")
        
        elif choice == '5':
            filename = input("백업 파일명 (엔터: diary_backup.txt): ").strip()
            if not filename:
                filename = 'diary_backup.txt'
            result = diary.export_to_text(filename)
            print(f"💾 {result}")
        
        elif choice == '0':
            print("\n👋 일기장을 종료합니다. 좋은 하루 되세요!")
            break
        
        else:
            print("❌ 올바른 번호를 선택해주세요.")

print("\n🎉 일기장 프로그램 데모가 완료되었습니다!")
print("실제 실행은 run_diary_app() 함수를 호출하세요.")

# 생성된 파일들 확인
files = ['diary_data.json', 'my_diary_backup.txt']
print("\n📁 생성된 파일들:")
for file in files:
    if os.path.exists(file):
        size = os.path.getsize(file)
        print(f"  ✅ {file} ({size} bytes)")
    else:
        print(f"  ❌ {file} (없음)")

## 프로젝트 완성 및 정리

### 🎯 구현된 기능

1. **일기 작성 및 저장**
   - JSON 형식으로 데이터 저장
   - 날짜별 일기 관리
   - 기분 상태 기록

2. **검색 및 조회**
   - 키워드 기반 검색
   - 날짜별 일기 조회
   - 전체 일기 목록

3. **통계 및 분석**
   - 일기 작성 통계
   - 기분 분포 분석
   - 단어 수 통계

4. **백업 및 내보내기**
   - 텍스트 파일로 백업
   - 읽기 쉬운 형식으로 변환

### 💡 활용된 개념

- **파일 입출력**: JSON 데이터 저장/로드
- **클래스 설계**: 일기장 매니저 클래스
- **데이터 구조**: 딕셔너리와 리스트 활용
- **문자열 처리**: 검색 및 텍스트 가공
- **날짜 처리**: datetime 모듈 활용

### 🚀 확장 가능한 기능

- 일기 수정/삭제 기능
- 태그 시스템
- 이미지 첨부
- 클라우드 동기화
- 웹 인터페이스
- 데이터 시각화

### 🏆 Chapter 9 학습 성과

파일 입출력의 기초부터 실제 활용까지 체계적으로 학습했습니다:

1. **기본 파일 조작** ✅
2. **CSV/JSON 처리** ✅  
3. **실전 프로젝트** ✅

다음 Chapter 10에서는 **예외처리**에 대해 학습하겠습니다!